In [2]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix
from sklearn import metrics


In [3]:
df = pd.read_csv("Dataset.csv", encoding = 'latin1')

In [4]:
#random sampling
random.seed(42)
sample_negatives_random= df.sample(frac=0.75)
sample_positives_random= df.sample(frac=0.25)
sample_negatives_random['Is_Fire'] = 0
sample_positives_random['Is_Fire'] = 1  
sample_random = pd.concat([sample_negatives_random,sample_positives_random])
sample_random = sample_random.sample(frac=1)
sample_random.to_csv("random_0.75.csv")

In [5]:
#systematic sampling
def systematic_sampling(df, steps, n):
  indexes = np.arange(0, len(df), step = steps)
  sample_negatives_systematic = df.iloc[indexes]
  return sample_negatives_systematic

sample_negatives_systematic = systematic_sampling(df, 3, 2)

In [6]:
#stratified_sampling
stratifies_sampling = df.groupby('Location__area_12', group_keys=False).apply(lambda x: x.sample(200, replace = True))

In [7]:
X = sample_random.iloc[:,41]
X

1114    1
561     0
1009    0
1550    0
841     1
       ..
1617    0
625     0
17      0
1422    0
448     0
Name: Is_Fire, Length: 1736, dtype: int64

In [8]:
new_df = sample_random
df

,OBJECTID1,File_tag,Provider,DateOnly,Date,Date_1,Time_2,Latitude_3,Longitude_3,Address_of_incident_4,...,HEIGHT,FOREST,Elevation,STATION_ID,CODE,NAME,WeatherStation,Monthly Avg Rainfall,Monthly Mean Max Temperature,Monthly Mean Solar Exposure
0,1,Powercor 18/107,Powercor,14/12/2018,14/12/2018,NaN,NaN,-38.729695,143.376872,375 Blue Johanna Rd. Johanna,...,2,1,237.341721,95855,WEEA,WEEAPROINAH,NaN,117.3,25.7,22.1
1,2,PowerCor 19/212,PowerCor,24/01/2020,1/24/20 0:00,43854.00000,0.667361,-38.754127,143.667405,"27 Pascoe St, Apollo Bay VIC 3233",...,6,0,7.690627,94842,YCTY,CAPE OTWAY,NaN,129.4,26.9,21.8
2,3,Powercor 18/143,Powercor,23/01/2019,23/01/2019,NaN,NaN,-38.716012,143.729673,"5750 GREAT OCEAN ROAD,PETTICOAT CREEK,VIC,3233",...,6,0,16.306395,95855,WEEA,WEEAPROINAH,NaN,12.6,28.4,25.6
3,4,Ausnet 17/107,Ausnet,20/02/2018,2/20/18 16:39,43151.69375,43151.693750,-38.669141,145.615890,"18 SPEAR STREET, CAPE PATERSON VIC 3995",...,6,0,25.977764,95881,YWON,WONTHAGGI,NaN,1.2,27,20.8
4,5,Ausnet 17/109,Ausnet,23/02/2018,2/23/18 21:57,43154.91458,43154.914580,-38.703816,145.819190,"90 JUPITER BOULEVARD, VENUS BAY VIC 3956",...,6,0,9.772949,95881,YWON,WONTHAGGI,NaN,1.2,27,20.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,1732,Powercor 16/54,Powercor,18/11/2016,11/18/16 0:00,42692.00000,0.413889,-34.183816,142.129660,NaN,...,6,0,40.722187,94693,YMIA,MILDURA,NaN,36.4,20.8,19.5
1732,1733,Powercor 16/196,Powercor,20/04/2017,4/20/17 0:00,42845.00000,0.571528,-34.199148,142.165160,NaN,...,6,0,46.991577,94693,YMIA,MILDURA,NaN,142.7,20.8,10.8
1733,1734,Powercor 16/203,Powercor,10/05/2017,5/10/17 0:00,42865.00000,0.984722,-34.203372,142.140880,NaN,...,6,0,54.408455,94693,YMIA,MILDURA,NaN,29.6,17.2,8.3
1734,1735,PowerCor 19/288,PowerCor,23/04/2020,4/23/20 0:00,43944.00000,0.789583,-34.206091,142.177030,2 Eleventh Street (corner Cowra Avenue)Mildura...,...,6,0,52.112083,94693,YMIA,MILDURA,NaN,145.6,19.3,9.8


In [9]:
new_df = new_df.drop(['OBJECTID1','File_tag','Provider','Date','Address_of_incident_4','CODE','NAME','ID_of__nearest__asset_5',
             'ID_of_nearest__polyphase__line_','Type_of_primary_asset__involved','Kind_of_fire_start_as_per_Claus',
             'Location__area_12','Fire__danger__rating_13','DNSP_record_number_14','OSIRIS___ESV__reference__number',
             'Fault__description_16','Overhead_conductors','BUFF_DIST','FOR_CODE','STATION_ID','CODE','NAME','WeatherStation',
             'Voltage_of_line_in_which__fire_','Network_categorisation_8','Phase_s__of__line_or__transform','FOR_TYPE','FOR_CAT'], axis = 1)

In [10]:
new_df = new_df.dropna()

In [11]:
"""
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
X = onehotencoder.fit_transform(new_df.Voltage_of_line_in_which__fire_.values.reshape(-1,1))
dfOneHot = pd.DataFrame(X) 
new_df = pd.concat([new_df, dfOneHot], axis=1)
#droping the country column 
new_df= new_df.drop(['Voltage_of_line_in_which__fire_'], axis=1)
"""
new_df.shape

(1625, 16)

In [12]:
X = new_df.iloc[:,1:15]
y = new_df.iloc[:,15]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [13]:
#Logistic Regression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)
names = ['0', '1']

In [14]:
classification_report(y_test, y_pred, target_names = names)

C:\Users\monit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\monit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\monit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0       0.76      1.00      0.86       407\n           1       0.00      0.00      0.00       130\n\n    accuracy                           0.76       537\n   macro avg       0.38      0.50      0.43       537\nweighted avg       0.57      0.76      0.65       537\n'